In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\IT STORE\\Desktop\\ML-Projects\\Text-Summarizer'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransfomationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [3]:
from src.constants import *
from src.utils import read_yaml, create_directories


In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
        
    def get_data_transformation_config(self) -> DataTransfomationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config = DataTransfomationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name,
        )
        return data_transformation_config
        

In [5]:
import os 
from src.logger import logging
from transformers import BartTokenizer
from datasets import load_dataset, load_from_disk


[2024-06-08 19:58:39,852: INFO: utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2024-06-08 19:58:39,853: INFO: utils: NumExpr defaulting to 8 threads.]


In [8]:
class DataTransformation:
    def __init__(self, config: DataTransfomationConfig):
        self.config = config
        self.tokenizer = BartTokenizer.from_pretrained(str(config.tokenizer_name))
        
    def convert_examples_to_features(self, example_batch):
        input_encoding = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)
        
        with self.tokenizer.as_target_tokenizer():
            target_encoding = self.tokenizer(example_batch['summary'], max_length=128, truncation=True)
            
        return {
            'input_ids': input_encoding['input_ids'],
            'attention_mask': input_encoding['attention_mask'],
            'decoder_input_ids': target_encoding['input_ids'],
            'decoder_attention_mask': target_encoding['attention_mask'],
        }
    
    def convert(self):
        # Ensure the paths are converted to strings
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(str(self.config.root_dir), "samsum_dataset"))


In [9]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-06-08 20:06:48,895: INFO: utils: yaml file: config\config.yaml loaded successfully.]
[2024-06-08 20:06:48,897: INFO: utils: yaml file: params.yaml loaded successfully.]
[2024-06-08 20:06:48,901: INFO: utils: Created directory at: artifacts]
[2024-06-08 20:06:48,902: INFO: utils: Created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

c:\Users\IT STORE\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]